In [1]:
# Importujeme potřebné knihovny pro práci se soubory (os),
# numerickými poli (numpy), načítáním a zpracováním obrázků (skimage),
# kódováním štítků (to_categorical), augmentací dat (ImageDataGenerator)
# a stavbou neuronové sítě (Keras/TensorFlow).

import os
import numpy as np
import zipfile
from skimage import io, transform
from skimage.color import rgb2gray
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

In [2]:
# Zde si stáhněte dataset 1: https://www.kaggle.com/datasets/kaggleashwin/vehicle-type-recognition#
# Zde si stáhněte dataset 2: https://www.kaggle.com/datasets/mohamedmaher5/vehicle-classification/data

In [ ]:
# Rozbalení všech ZIP souborů ve složce ./data

# Do data_dir vložte cestu ke staženým datasetům

data_dir = "./data"

for file in os.listdir(data_dir):
    if file.endswith(".zip"):
        file_path = os.path.join(data_dir, file)
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
        print(f"Rozbaleno: {file}")


In [ ]:
# Definuje seznamy adresářů s trénovacími a testovacími daty
# a nastavuje cílovou velikost (rozlišení) obrázků na 100×100 pixelů.

data1 = "data1" #Cesta k rozbalenému souboru data1
data2 = "data2" #Cesta k rozbalenému souboru data2


train_dirs = [data1, data2]
test_dirs  = [data1, data2]
image_size = (100, 100)

In [ ]:
# Načte názvy tříd z tréninkových složek (seřazené)
# a vytvoří slovník label_dict, který mapuje název třídy a číselný index.

train_classes = sorted({cls
                        for d in train_dirs
                        for cls in os.listdir(d) if os.path.isdir(os.path.join(d, cls))})
label_dict = {cls: idx for idx, cls in enumerate(train_classes)}

In [ ]:
# Pro každý testovací adresář zjistí, jaké třídy obsahuje,
# a potom vypočítá průnik (společné třídy) mezi všemi sadami.

all_sets = [set(train_classes)]
for d in test_dirs:
    all_sets.append({cls for cls in os.listdir(d) if os.path.isdir(os.path.join(d, cls))})
common_classes = sorted(set.intersection(*all_sets))

In [ ]:
# Definuje funkci load_images(dirs, classes, label_dict),
# která projde všechny soubory daných tříd v zadaných složkách,
# načte obrázek, změní jeho velikost, převede na odstíny šedi,
# přidá do seznamu X a odpovídající label (y) podle label_dict.

def load_images(dirs, classes, label_dict):
    if isinstance(dirs, str):
        dirs = [dirs]
    X, y = [], []
    for cls in classes:
        for d in dirs:
            class_path = os.path.join(d, cls)
            if not os.path.isdir(class_path):
                continue
            for fname in os.listdir(class_path):
                try:
                    img = io.imread(os.path.join(class_path, fname))
                    img = transform.resize(img, image_size, anti_aliasing=True)
                    img = rgb2gray(img)
                    X.append(img)
                    y.append(label_dict[cls])
                except Exception as e:
                    print(f"Chyba u {fname}: {e}")
    X = np.array(X).reshape(-1, *image_size, 1)
    y = np.array(y)
    return X, y

In [ ]:
# Načte trénovací (X_train, y_train) a testovací (X_test, y_test) data
# pomocí výše definované funkce load_images.

X_train, y_train = load_images(train_dirs, train_classes, label_dict)
X_test,  y_test  = load_images(test_dirs,  common_classes, label_dict)

In [ ]:
# Převede jednorozměrné labely y_train/y_test na one-hot
# pro vícetřídovou klasifikaci a vypíše tvary dat a seznam tříd.

Y_train = to_categorical(y_train, num_classes=len(train_classes))
Y_test  = to_categorical(y_test,  num_classes=len(train_classes))

print("Train:", X_train.shape, "| Test:", X_test.shape)
print("Trénuji na třídách:", train_classes)
print("Testuji jen na společných:", common_classes)

In [ ]:
# Vytvoří konvoluční neuronovou síť (Sequential) s vrstvami:
# dvěma Conv2D + MaxPooling, Dropout, Flatten, Dense + Dropout,
# a na závěr Dense s aktivací softmax pro klasifikaci do počtu tříd.
# Poté model zkompiluje s loss='categorical_crossentropy',
# optimalizátorem 'adam' a metrikou 'accuracy'.

model = Sequential([
    Conv2D(64, (3,3), activation='relu', input_shape=(*image_size,1)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.5),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.25),
    Dense(len(train_classes), activation='softmax'),
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
#  Nastaví augmentaci trénovacích dat pomocí ImageDataGenerator
# (oříznutí, posuny, zoom, horizontální převrácení),
# fitne na X_train a spustí trénink modelu na 50 epochách.

datagen = ImageDataGenerator(
    rotation_range=20, width_shift_range=0.1,
    height_shift_range=0.1, zoom_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)
model.fit(datagen.flow(X_train, Y_train, batch_size=64),
          epochs=50, verbose=1)

In [ ]:
# Vyhodnotí trénovaný model na trénovacích i testovacích datech
# a vytiskne přesnost (accuracy) pro obě sady.

train_score = model.evaluate(X_train, Y_train, verbose=0)
test_score  = model.evaluate(X_test,  Y_test,  verbose=0)
print(f"Train accuracy: {train_score[1]:.4f}")
print(f" Test accuracy: {test_score[1]:.4f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Předpovědi modelu (pravděpodobnosti)
y_pred_probs = model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

# Skutečné hodnoty ze vstupu
y_true = np.concatenate([np.argmax(y.numpy(), axis=1) for _, y in test_ds])

# Výpočet konfuzní matice
cm = confusion_matrix(y_true, y_pred)

# Zobrazení
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=train_classes, yticklabels=train_classes)
plt.xlabel("Predikovaná třída")
plt.ylabel("Skutečná třída")
plt.title("Konfuzní matice")
plt.show()


In [ ]:
import requests
from PIL import Image
from io import BytesIO

# URL obrázku – nahraď jiným, pokud chceš
url = "https://upload.wikimedia.org/wikipedia/commons/4/47/American_bulldog.jpg"

# Načtení obrázku
response = requests.get(url)
image = Image.open(BytesIO(response.content)).convert("RGB")
image = image.resize(image_size)

# Zobrazení obrázku
import matplotlib.pyplot as plt
plt.imshow(image)
plt.axis("off")
plt.title("Načtený obrázek z internetu")
plt.show()

# Předzpracování
img_array = tf.keras.preprocessing.image.img_to_array(image)
img_array = preprocess_input(img_array)
img_array = tf.expand_dims(img_array, axis=0)

# Predikce
prediction = model.predict(img_array)
predicted_label = train_classes[np.argmax(prediction)]

print(f"Predikovaná třída: {predicted_label}")
